# Copy image files to LoRA dataset

In [2]:
import boto3

def copy_files_within_bucket(bucket_name, source_folder, destination_folder):
    s3 = boto3.client('s3')

    # Get the list of objects in the source folder
    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=source_folder)

    # Iterate over the objects and copy them to the destination folder
    for obj in response['Contents']:
        source_key = obj['Key']
        destination_key = source_key.replace(source_folder, destination_folder)

        # Copy the object to the destination folder
        s3.copy_object(
            Bucket=bucket_name,
            CopySource={'Bucket': bucket_name, 'Key': source_key},
            Key=destination_key
        )

# Specify the bucket name, source folder, and destination folder
bucket_name = 'sagemaker-image-generation-data'
source_folder = 'textual_inversion_datasets/multi_layer_color/multi_layer_color_64'
destination_folder = 'lora_datasets/multi_layer_color/multi_layer_color_64'

# Call the function to copy files within the bucket
copy_files_within_bucket(bucket_name, source_folder, destination_folder)

# Make txt files

In [10]:
s3 = boto3.client('s3')

# List the objects in the S3 bucket folder
response = s3.list_objects_v2(Bucket=bucket_name, Prefix=destination_folder)

# Iterate through the objects
for obj in response['Contents']:
    # Get the file name and extension
    file_name = obj['Key'].split('/')[-1]
    file_extension = file_name.split('.')[-1]

    # Check if the object is an image file
    if file_extension.lower() in ['jpg', 'jpeg', 'png', 'gif']:
        # Create a new text file with the same name as the image file
        text_file_name = file_name.split('.')[0] + '.txt'
        text_content = "multi color, " + file_name.split('.')[0]
        
        # Upload the text content as a new object in the S3 bucket folder
        s3.put_object(Body=text_content, Bucket=bucket_name, Key=destination_folder + '/' + text_file_name)